# Laboratorio 2 - Informe

### Grupo 4:
     - S. Calvo C.I 5.711.417-7     
     - X. Iribarnegaray C.I 5.253.705-9
     - J. Simonelli C.I 5.405.358-4

## 1. Objetivos
El objetivo de este laboratorio es:
- Implementar el algoritmo Naive Bayes
- Aplicar herramientas de metodología
- Analizar los resultados obtenidos.

## 2. Diseño
### 2.1 Algoritmo

### Explicar Naive Bayes logaritmico, y presentar el otro analogo ###

In [1]:
import numpy as np
from naive_bayes import init

dataset, features, continuous_features, target = init()

def naive_bayes(dataset, target, features, instance, m):
    dataset_size = dataset.shape[0]
    prob_1 = dataset[target].value_counts()[1]/dataset_size
    prob_0 = dataset[target].value_counts()[0]/dataset_size
    
    sum_1 = np.log(prob_1)
    sum_0 = np.log(prob_0)
    
    for feature in features:
        examples = dataset.loc[dataset[feature] == instance[feature]][target].value_counts()
        
        # if no instances with a specific target value is found, the get method will return 0
        count_1 = examples.get(1, default=0)
        count_0 = examples.get(0, default=0)
        
        feature_range = len(dataset[feature].value_counts())
        
        numerator_1 = count_1 + (m / feature_range)
        numerator_0 = count_0 + (m / feature_range)

        # sum of sequence
        sum_1 += np.log( numerator_1 / (dataset[target].value_counts()[1] + m) )
        sum_0 += np.log( numerator_0 / (dataset[target].value_counts()[0] + m) )
        
    # argmax
    if ( sum_1 > sum_0):
        return 1
    else:
        return 0


### 2.3 Evaluación

In [2]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def test_instances(X_train, y_train, X_test, y_test, features, m):
    y_pred = []
    train_ds = X_train.copy()
    train_ds[target] = y_train
    for i in range(0, X_test.shape[0]):
        instance = X_test.iloc[i]
        y_pred.append(naive_bayes(train_ds, target, features, instance, m))
        
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred,zero_division=1)
    recall = recall_score(y_test, y_pred,zero_division=1)
    f1 = f1_score(y_test, y_pred,zero_division=1)
    
    return accuracy*100, precision*100, recall*100, f1*100

## 3. Experimentación

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

kbins = KBinsDiscretizer(n_bins=50, encode='ordinal', strategy='kmeans')

# Discretize continuous features
dataset[continuous_features] = kbins.fit_transform(dataset[continuous_features])

# Prepare the feature matrix and target
X = dataset.drop([target, 'pidnum'], axis=1)
y = dataset[target]

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

m_values = [1, 10, 100, 1000]

results = pd.DataFrame(columns=['m', 'Accuracy', 'Precision', 'Recall', 'F1'])

for i in range(0, len(m_values)):
    m = m_values[i]
    accuracy, precision, recall, f1 = test_instances(X_train, y_train, X_test, y_test, features, m)
    
    results.loc[i] = {'m': m, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1}

print(results)


Explicar por qué elegimos los parametros que pusimos en el kbins

#### KBins and Feature Selection preprocessing ####

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from naive_bayes import test_instances, init
from sklearn.feature_selection import SelectKBest, chi2

dataset, features, continuous_features, target = init()

kbins = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='kmeans')
dataset[continuous_features] = kbins.fit_transform(dataset[continuous_features])

X = dataset.drop(target, axis=1)
y = dataset[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

''' selector = SelectKBest(chi2, k=2)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test) '''


print('Naive Bayes logarítmico')
print('m=1: ',test_instances(X_train, y_train, X_test, y_test,1))
print('m=10: ',test_instances(X_train, y_train, X_test, y_test,10))
print('m=100: ',test_instances(X_train, y_train, X_test, y_test,100))
print('m=1000: ',test_instances(X_train, y_train, X_test, y_test,1000))


In [ ]:
from naive_bayes import init, test_instances
from sklearn.feature_selection import SelectKBest, chi2

dataset, features, continuous_features, target = init()

kbins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')
dataset[continuous_features] = kbins.fit_transform(dataset[continuous_features])

X = dataset.drop(target, axis=1)
y = dataset[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

selector = SelectKBest(chi2, k=2)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)



new_features = 



In [ ]:
## Analyze continuous features value distribution
dataset[continuous_features].hist(bins=30, figsize=(12, 8), color='skyblue', edgecolor='black')
plt.suptitle('Distribución de Valores de Atributos Continuos')
plt.show()